In [6]:
# Original
# Only consider the first 500 (FP: 1454, Precision: 0.0352)
# DATE_TIME already excluded
# Reason that DATE_TIME is not necessary:
# 1. DATE_TIME cannot associate to people names (don't know who said the date_time)
# 2. DATE_TIME can be relevant to math problem descriptions if finally applied to PLUS tutoring team.
# 3. ...?
# Check Recall
import ast
from typing import List, Tuple

# Define the type alias for PII entity
pii_entity = Tuple[int, str, str, Tuple[int, int]]

def load_entities(file_path: str) -> List[pii_entity]:
    entities = []
    with open(file_path, 'r') as file:
        for line in file:
            entity = ast.literal_eval(line.strip())
            entities.append(entity)
    return entities

# Load true entities
true_entities = load_entities('data/pii_entities.txt')

def calculate_metrics(input_detected_entities):
    detected_entities = load_entities(input_detected_entities)
    # Exclude the entity type from the comparison
    true_set = set((i, start, end) for i, _, _, (start, end) in true_entities)
    detected_set = set((i, start, end) for i, _, _, (start, end) in detected_entities)

    tp = true_set & detected_set
    fp = detected_set - true_set
    fn = true_set - detected_set

    tp_count = len(tp)
    fp_count = len(fp)
    fn_count = len(fn)

    precision = tp_count / (tp_count + fp_count) if (tp_count + fp_count) > 0 else 0
    recall = tp_count / (tp_count + fn_count) if (tp_count + fn_count) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    f5_score = (1 + 5**2) * (precision * recall) / ((5**2 * precision) + recall) if (precision + recall) > 0 else 0

    print(f"True Positives (TP): {tp_count}")
    print(f"False Positives (FP): {fp_count}")
    print(f"False Negatives (FN): {fn_count}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1_score:.4f}")
    print(f"F5-Score: {f5_score:.4f}")


In [11]:
calculate_metrics('output/pii_detected_lg.txt')

True Positives (TP): 3442
False Positives (FP): 19606
False Negatives (FN): 1429
Precision: 0.1493
Recall: 0.7066
F1-Score: 0.2466
F5-Score: 0.6179


In [10]:
calculate_metrics('output/pii_detected_trf.txt')

True Positives (TP): 4044
False Positives (FP): 15757
False Negatives (FN): 827
Precision: 0.2042
Recall: 0.8302
F1-Score: 0.3278
F5-Score: 0.7427


In [14]:
calculate_metrics('others/pii_detected_trf_threshold.txt')

True Positives (TP): 4034
False Positives (FP): 12559
False Negatives (FN): 837
Precision: 0.2431
Recall: 0.8282
F1-Score: 0.3759
F5-Score: 0.7580


In [13]:
calculate_metrics('output/pii_detected_trf_filtered.txt')

True Positives (TP): 4044
False Positives (FP): 12629
False Negatives (FN): 827
Precision: 0.2425
Recall: 0.8302
F1-Score: 0.3754
F5-Score: 0.7594


In [ ]:
# TODO: Run previous and this code chunk. No need to run following code chunks.
calculate_metrics('output/pii_detected_gpt.txt')

In [3]:
# calculate_metrics('others/pii_entities_detected_updated.txt')

True Positives (TP): 4359
False Positives (FP): 18683
False Negatives (FN): 512
Precision: 0.1892
Recall: 0.8949
F1-Score: 0.3123
F5-Score: 0.7826


In [4]:
# calculate_metrics('others/pii_entities_detected_updated2.txt')

True Positives (TP): 4359
False Positives (FP): 15528
False Negatives (FN): 512
Precision: 0.2192
Recall: 0.8949
F1-Score: 0.3521
F5-Score: 0.8000


In [5]:
# Note for Jack: Deal with overlapping entities.
def check_overlapping_entities(input_detected_entities):
    entities = load_entities(input_detected_entities)
    # Sort entities by index and start position
    entities_sorted = sorted(entities, key=lambda x: (x[0], x[3][0]))
    overlaps = []
    
    for i in range(len(entities_sorted) - 1):
        current_entity = entities_sorted[i]
        next_entity = entities_sorted[i + 1]
        
        # Check if the current entity overlaps with the next one
        if current_entity[0] == next_entity[0] and current_entity[3][1] > next_entity[3][0]:
            overlaps.append((current_entity, next_entity))
    
    # Print the overlapping entities
    if overlaps:
        print(f"Found {len(overlaps)} overlapping entities:")
        for overlap in overlaps:
            print(f"Overlap between: {overlap[0]} and {overlap[1]}")
    else:
        print("No overlapping entities found.")

check_overlapping_entities('output/pii_entities_detected_updated2.txt')

Found 149 overlapping entities:
Overlap between: (5, 'https://www.greatplacetowork.com/resources/blog/why-is-diversity-inclusion-in-the-workplace-important', 'PERSON', (4150, 4251)) and (5, 'https://www.greatplacetowork.com/resources/blog/why-is-diversity-inclusion-in-the-workplace-important', 'URL', (4150, 4251))
Overlap between: (123, 'https://cyberleninka.ru/article/n/stremlenie-k-spravedlivomu-sotrudnichestvu-kak- motiv-ekonomicheskogo-povedeniya', 'PERSON', (7742, 7856)) and (123, 'https://cyberleninka.ru/article/n/stremlenie-k-spravedlivomu-sotrudnichestvu-kak-', 'URL', (7742, 7823))
Overlap between: (1199, 'Joseph  with', 'PERSON', (1680, 1692)) and (1199, 'Joseph  with Jesus', 'PERSON', (1680, 1698))
Overlap between: (1631, 'Natalia', 'PERSON', (13, 20)) and (1631, 'Natalia Andrea Ibarra Villarreal', 'PERSON', (13, 45))
Overlap between: (1799, 'https://www.aljazeera.com/news/2020/05/forty-percent-nigerians-live-poverty-line-report-', 'URL', (385, 473)) and (1799, 'https://www.a